In [1]:
# from simcse import SimCSE
from entity_retrieval import surface_index_memory
# from your_module import convert_normed_to_s_expression  # Đổi 'your_module' thành tên file của bạn
# from simcse import SimCSE
from itertools import product
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import ujson
import json
import time

In [2]:


surface_index = surface_index_memory.EntitySurfaceIndexMemory(
    "vi_entity_list_file_wikidata_complete_all_mention", "vi_entity_surface_map_file_wikidata_complete_all_mention","vi_wiki_complete_all_mention" )

INFO:entity_retrieval.surface_index_memory:Loading entity vocabulary from disk.
INFO:entity_retrieval.surface_index_memory:Loading surfaces from disk.
INFO:entity_retrieval.surface_index_memory:Done initializing surface index.


In [38]:
import requests

def find_wikidata_entity(label: str, language: str = "vi"):
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": label,
        "language": language,
        "type": "item",
        "format": "json"
    }

    try:
        response = requests.get(url, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        print(f"Lỗi truy vấn Wikidata: {e}")
        return None

    if "search" in data and data["search"]:
        results = [(item["id"], item.get("label", ""), item.get("description", "")) for item in data["search"]]

        # Lọc bỏ các kết quả không có nhãn hợp lệ (None hoặc số)
        results = [(id, str(label), desc) for id, label, desc in results if isinstance(label, str)]

        # # Ưu tiên nhãn khớp chính xác trước
        # results.sort(key=lambda x: (x[1].lower() != label.lower(), not x[2]), reverse=True)

        exact_matches = [item[0] for item in results if item[1].lower() == label.lower()]
        if exact_matches:
            return exact_matches

    return search_wikidata_pages(label)


def search_wikidata_pages(label: str):
    """
    Tìm các trang Wikidata có chứa thực thể liên quan đến nhãn.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": label,
        "format": "json"
    }

    try:
        response = requests.get(url, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        print(f"Lỗi truy vấn trang Wikidata: {e}")
        return None

    if "query" in data and "search" in data["query"]:
        return [f"{item['title']}" for item in data["query"]["search"]]

    return None
def search_wikidata_entities(label: str, language: str = "vi"):
    """
    Tìm các thực thể Wikidata có mã Q cụ thể liên quan đến nhãn.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": label,
        "language": language,
        "type": "item",  # Chỉ tìm thực thể (bỏ qua quan hệ P)
        "format": "json"
    }

    try:
        response = requests.get(url, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        print(f"Lỗi truy vấn Wikidata: {e}")
        return None

    if "search" in data:
        return [item["id"] for item in data["search"]]  # Lấy mã thực thể Q

    return None
def search_wikidata_relations(label: str, language: str = "vi"):
    """
    Tìm các mã quan hệ (P...) trên Wikidata dựa vào nhãn.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "search": label,
        "language": language,
        "type": "property",  # Chỉ tìm quan hệ (P...)
        "format": "json"
    }

    try:
        response = requests.get(url, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        print(f"Lỗi truy vấn Wikidata: {e}")
        return None

    if "search" in data:
        return [item["id"] for item in data["search"]]  # Lấy mã quan hệ P

    return None
# Ví dụ tìm mã thực thể cho "Mary Lou Retton"
print(search_wikidata_entities("vehicle"))

# Ví dụ sử dụng
print(search_wikidata_relations("cho tác phẩm"))

['Q42889', 'Q66424155', 'Q48833186', 'Q7918414', 'Q7918409', 'Q28094310', 'Q66424271']
['P1686']


In [32]:
import json
import requests

def evaluate_wikidata_matching(test_file):
    with open(test_file, "r", encoding="utf-8") as f:
        test_data = json.load(f)
    test_data = test_data[:250]
    total_entities = 0
    exact_match_count = 0
    contained_match_count = 0
    
    for data in test_data:
        gold_entity_map = data.get("gold_relation_map", {})
        for gold_id, label in gold_entity_map.items():

            total_entities += 1
            label = str(label)
            result = search_wikidata_relations(label)[:10]
            
            if not result:
                continue
            # Kiểm tra nếu mã đầu tiên trong danh sách khớp
            if result[0] == gold_id:
                exact_match_count += 1
            
            # Kiểm tra nếu danh sách có chứa mã gold_id
            if gold_id in result:
                contained_match_count += 1
    
    exact_match_ratio = (exact_match_count / total_entities) * 100 if total_entities > 0 else 0
    contained_match_ratio = (contained_match_count / total_entities) * 100 if total_entities > 0 else 0
    
    print(f"Tổng số thực thể: {total_entities}")
    print(f"Tỉ lệ khớp chính xác: {exact_match_ratio:.2f}%")
    print(f"Tỉ lệ chứa mã thực thể đúng: {contained_match_ratio:.2f}%")

# Gọi hàm đánh giá với tập test
evaluate_wikidata_matching("Data/LC-QuAD2.0/label_map/LC-QuAD2.0_test.json")


Tổng số thực thể: 422
Tỉ lệ khớp chính xác: 98.34%
Tỉ lệ chứa mã thực thể đúng: 100.00%


In [3]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")


def get_embedding(text):
    """Trích xuất vector embedding từ mô hình"""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)

    # Mean pooling
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings



c:\Users\TOPU\anaconda3\envs\py38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def find_entity(label, label_map, facc1_index, top_k=20, similarity_threshold=0.2):
    """
    Tìm thực thể hoặc quan hệ từ gold maps, SimCSE hoặc FACC1.
    """
    label_lower = label.lower()

    # Nếu có trong label_map, trả về ngay
    if label_map and label_lower in label_map:
        return label_map[label_lower]

    # Lấy embedding cho label
    label_embedding = get_embedding(label_lower).reshape(1, -1)

    if label_map:
        label_keys = list(label_map.keys())
        # Lấy embedding từ giá trị (tên thực thể) thay vì key
        label_embeddings = np.array([get_embedding(label_map[k]) for k in label_keys]).squeeze(1)

        # Normalize trước khi tính cosine similarity
        label_embedding = normalize(label_embedding, axis=1)
        label_embeddings = normalize(label_embeddings, axis=1)

        # Tính cosine similarity
        similarities = cosine_similarity(label_embedding, label_embeddings).flatten()

        # Chọn thực thể gần nhất
        merged_list = list(zip(label_keys, similarities))
        sorted_list = sorted(merged_list, key=lambda x: x[1], reverse=True)

        if sorted_list and sorted_list[0][1] > 0.2:
            return label_map[sorted_list[0][0]]
        
    facc1_cand_entities = facc1_index.get_indexrange_entity_el_pro_one_mention(label_lower, top_k=50)
    print(facc1_cand_entities)
    if facc1_cand_entities:
        temp = []
        for key in list(facc1_cand_entities.keys())[1:]:
            if facc1_cand_entities[key] >= 0.001:
                temp.append(key)
        if len(temp) > 0:
            label = [list(facc1_cand_entities.keys())[0]]+temp
        else:
            label = list(facc1_cand_entities.keys())[0]

    return label

In [13]:
simcse_model = model  # Mô hình SimCSE của bạn
facc1_index = surface_index
label_map = {}

In [18]:
find_entity("Jean Umansky", label_map,facc1_index)

OrderedDict([('Q6171615', 1.0)])


'Q6171615'

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModel
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.preprocessing import normalize

# # Load model & tokenizer
# tokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
# model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")


# def get_embedding(text):
#     """Trích xuất vector embedding từ mô hình"""
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
#     with torch.no_grad():
#         outputs = model(**inputs)

#     # Mean pooling
#     embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
#     return embeddings


# gold_entity_map = {
#     "Q1069938": "Vår Frelsers gravlund",
#     "Q36661": "Henrik Ibsen"
# }

# label_lower = "Nghĩa trang Var Frelsers".lower()  # CHUẨN HÓA

# # Lấy embedding cho label
# label_embedding = get_embedding(label_lower).reshape(1, -1)

# if gold_entity_map:
#     label_keys = list(gold_entity_map.keys())
#     # Lấy embedding từ giá trị (tên thực thể) thay vì key
#     label_embeddings = np.array([get_embedding(gold_entity_map[k]) for k in label_keys]).squeeze(1)

#     # Normalize trước khi tính cosine similarity
#     label_embedding = normalize(label_embedding, axis=1)
#     label_embeddings = normalize(label_embeddings, axis=1)

#     # Tính cosine similarity
#     similarities = cosine_similarity(label_embedding, label_embeddings).flatten()

#     # In giá trị cosine similarity từng thực thể
#     for key, sim in zip(label_keys, similarities):
#         print(f"Similarity với {gold_entity_map[key]} ({key}): {sim}")

#     # Chọn thực thể gần nhất
#     merged_list = list(zip(label_keys, similarities))
#     sorted_list = sorted(merged_list, key=lambda x: x[1], reverse=True)

#     if sorted_list and sorted_list[0][1] > 0.2:
#         print("Kết quả:", gold_entity_map[sorted_list[0][0]])


Similarity với Vår Frelsers gravlund (Q1069938): 0.4353713095188141
Similarity với Henrik Ibsen (Q36661): 0.32842618227005005
Kết quả: Vår Frelsers gravlund


In [16]:
def is_valid_expression(expr):
    """Kiểm tra tính hợp lệ của biểu thức bằng cách đếm số ngoặc mở và đóng."""
    count = 0
    for char in expr:
        if char == '(':
            count += 1
        elif char == ')':
            count -= 1
        if count < 0:
            return False  # Gặp ngoặc đóng trước ngoặc mở
    return count == 0

def fix_unbalanced_parentheses(expr):
    """Loại bỏ ngoặc đóng dư nếu có."""
    while not is_valid_expression(expr) and expr.endswith(')'):
        expr = expr[:-1]
    return expr

In [17]:

def find_entity_or_relation(label, label_map, simcse_model, facc1_index, top_k=50, similarity_threshold=0.2):
    """
    Tìm mã tương ứng cho thực thể hoặc quan hệ từ gold maps, SimCSE hoặc FACC1.
    Nếu label_map rỗng, sẽ bỏ qua bước similarity và chuyển thẳng sang tra cứu FACC1.
    """
    search = True
    label_lower = label.lower()
    # Nếu label_map không rỗng, thử tra cứu qua label_map và similarity
    if label_map:
        if label_lower in label_map:
            return label_map[label_lower]
        similarities = simcse_model.similarity([label_lower], list(label_map.keys()))
        if list(label_map.keys()):
            merged_list = list(zip(label_map.keys(), similarities[0]))
            sorted_list = sorted(merged_list, key=lambda x: x[1], reverse=True)
            if sorted_list and sorted_list[0][1] > similarity_threshold:
                return label_map[sorted_list[0][0]]
    
    # Nếu không có dữ liệu từ label_map, chuyển thẳng sang tra cứu FACC1
    try:
        print(label)
        facc1_candidates = facc1_index.get_indexrange_entity_el_pro_one_mention(label, top_k=top_k)
    except Exception as e:
        # Nếu có lỗi xảy ra trong quá trình tra cứu FACC1, log lỗi và trả về label gốc
        print(f"Lỗi khi truy xuất FACC1 cho label '{label}': {e}")
        return label

    if facc1_candidates:
        keys = list(facc1_candidates.keys())
        if not keys:
            return label
        try:
            # Lấy candidate đầu tiên và các candidate có điểm >= 0.001
            temp = [key for key in keys[1:] if facc1_candidates[key] >= 0.001]
            return [keys[0]] + temp if temp else keys[0]
        except Exception as e:
            print(f"Lỗi khi xử lý kết quả FACC1 cho label '{label}': {e}")
            return label

    return label

# def invert_map(original_map):
#     """Đảo ngược key-value trong map để tìm mã từ tên."""
#     return {v.lower(): k for k, v in original_map.items()}

def parse_nsexpr(expr):
    """
    Chuyển chuỗi biểu thức thành cây cấu trúc dạng nested list.
    Hàm này dùng duyệt ký tự, khi gặp '(' sẽ tìm phần con cho đến khi khớp với ')',
    và giữ nguyên nội dung trong ngoặc vuông.
    """
    tokens = []
    i = 0
    while i < len(expr):
        if expr[i].isspace():
            i += 1
        elif expr[i] == '(':
            # Tìm phần con của biểu thức trong ngoặc đơn
            count = 1
            j = i + 1
            while j < len(expr) and count > 0:
                if expr[j] == '(':
                    count += 1
                elif expr[j] == ')':
                    count -= 1
                j += 1
            # Đệ quy phân tích phần con (loại bỏ ngoặc bao ngoài)
            subtree = parse_nsexpr(expr[i+1:j-1])
            tokens.append(subtree)
            i = j
        elif expr[i] == '[':
            # Giữ nguyên nội dung trong ngoặc vuông
            j = expr.find(']', i)
            if j == -1:
                return ""
                raise ValueError("Không tìm thấy dấu ']' kết thúc.")
                
            token = expr[i:j+1].strip()
            tokens.append(token)
            i = j + 1
        else:
            # Đọc một token cho đến khi gặp khoảng trắng hoặc ngoặc
            j = i
            while j < len(expr) and (not expr[j].isspace()) and expr[j] not in ['(', ')']:
                j += 1
            tokens.append(expr[i:j])
            i = j
    return tokens

def collect_labels(tree):
    """
    Duyệt cây cấu trúc (nested list) để thu thập các nhãn của quan hệ và thực thể.
    Giả sử:
      - Biểu thức JOIN có dạng: ["JOIN", relation_part, entity_part]
      - Phần relation_part: nếu là list và bắt đầu bằng "R", thì phần thứ hai chứa nhãn quan hệ (dạng "[ label ]"). 
        Nếu là chuỗi dạng "[ label ]" thì đó cũng là nhãn quan hệ.
      - Phần entity_part: nếu là chuỗi dạng "[ label ]" thì đó là nhãn thực thể, nếu là list thì xử lý đệ quy.
      - Biểu thức AND sẽ có nhiều biểu thức con.
    """
    relations = []
    entities = []
    
    if isinstance(tree, list) and tree:
        # Nếu token đầu tiên là JOIN hoặc AND
        op = tree[0]
        if isinstance(op, str):
            op_upper = op.upper()
        else:
            op_upper = ""
        
        if op_upper == "JOIN":
            # Xử lý phần quan hệ
            if len(tree) >= 2:
                rel_part = tree[1]
                # Nếu là list dạng [ "R", "[ label ]" ]
                if isinstance(rel_part, list) and len(rel_part) >= 2 and isinstance(rel_part[0], str) and rel_part[0].upper() == "R":
                    token = rel_part[1]
                    if isinstance(token, str) and token.startswith('[') and token.endswith(']'):
                        rel_label = token[1:-1].strip()
                        relations.append(rel_label)
                # Nếu là chuỗi dạng "[ label ]"
                elif isinstance(rel_part, str) and rel_part.startswith('[') and rel_part.endswith(']'):
                    rel_label = rel_part[1:-1].strip()
                    relations.append(rel_label)
                else:
                    # Nếu không đúng định dạng, duyệt đệ quy
                    sub_rel, sub_ent = collect_labels(rel_part)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
            # Xử lý phần thực thể
            if len(tree) >= 3:
                ent_part = tree[2]
                if isinstance(ent_part, list):
                    sub_rel, sub_ent = collect_labels(ent_part)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
                elif isinstance(ent_part, str) and ent_part.startswith('[') and ent_part.endswith(']'):
                    ent_label = ent_part[1:-1].strip()
                    entities.append(ent_label)
        elif op_upper == "AND":
            # Với AND, duyệt tất cả các phần con
            for sub in tree[1:]:
                sub_rel, sub_ent = collect_labels(sub)
                relations.extend(sub_rel)
                entities.extend(sub_ent)
        else:
            # Nếu không phải JOIN hay AND, duyệt tất cả các phần tử nếu chúng là list
            for elem in tree:
                if isinstance(elem, list):
                    sub_rel, sub_ent = collect_labels(elem)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
    return relations, entities



def extract_entities_and_relations(normed_expr):

    if not normed_expr or len(normed_expr) == 0:  # Kiểm tra nếu normed_expr rỗng
        return [], []
    
    if normed_expr[0] != "(":
        return [], []
    
    tree = parse_nsexpr(normed_expr)
    if tree is None:
        return [], []  # Trả về danh sách rỗng nếu parse thất bại
    
    return collect_labels(tree)



In [18]:
def convert_normed_to_s_expression(normed_expr, gold_relation_map, gold_entity_map, simcse_model, facc1_index):
    """
    Chuyển đổi từ normed_sexpression sang s_expression.
    Sau khi trích xuất các nhãn quan hệ và thực thể từ normed_expr,
    ta lấy danh sách các mã ứng viên cho mỗi nhãn và tạo hoán vị giữa các cặp ứng viên đó.
    Kết quả trả về là một danh sách các s_expression khả dĩ.
    """
    # Đảo ngược map để lấy mã từ tên
    # inverted_relation_map = invert_map(gold_relation_map)
    # inverted_entity_map = invert_map(gold_entity_map)
    normed_expr = fix_unbalanced_parentheses(normed_expr)
    # Trích xuất các nhãn quan hệ và thực thể từ biểu thức
    relations, entities = extract_entities_and_relations(normed_expr)
    
    # Tạo mapping từ token xuất hiện trong biểu thức sang danh sách các ứng viên mã.
    # Ví dụ: token_str = "[ author ]"
    candidate_map = {}
    
    for rel in relations:
        token = f'[ {rel} ]'
        candidate = find_entity_or_relation(rel, gold_relation_map, simcse_model, facc1_index)
        # Nếu candidate không phải danh sách, chuyển nó thành danh sách để tạo hoán vị
        if not isinstance(candidate, list):
            candidate = [candidate]
        candidate_map[token] = candidate
        
    for ent in entities:
        token = f'[ {ent} ]'
        candidate = find_entity_or_relation(ent, gold_entity_map, simcse_model, facc1_index)
        if not isinstance(candidate, list):
            candidate = [candidate]
        candidate_map[token] = candidate
    
    # Nếu không có token nào cần thay thế, trả về biểu thức gốc
    if not candidate_map:
        return [normed_expr]
    
    # Lấy danh sách các token và danh sách các danh sách ứng viên tương ứng
    tokens = list(candidate_map.keys())
    candidate_lists = [candidate_map[token] for token in tokens]
    
    # Tạo tất cả các hoán vị ứng viên (Cartesian product)
    all_combinations = list(product(*candidate_lists))
    
    s_expressions = []
    for comb in all_combinations:
        temp_expr = normed_expr
        # Với mỗi token, thay thế bằng ứng viên tương ứng theo hoán vị
        for token, replacement in zip(tokens, comb):
            temp_expr = temp_expr.replace(token, replacement)
        s_expressions.append(temp_expr)
    
    return s_expressions

In [19]:
class SExpressionParser:
    def __init__(self):
        self.var_counter = 1  # Đếm số biến trung gian (?X1, ?X2, ...)

    def get_new_var(self):
        """Tạo biến trung gian mới."""
        var_name = f"?X{self.var_counter}"
        self.var_counter += 1
        return var_name

    def parse_s_expr(self, s_expr):
        """Chuyển đổi S-Expression thành danh sách lồng nhau."""
        s_expr = re.sub(r'\(', ' ( ', s_expr)
        s_expr = re.sub(r'\)', ' ) ', s_expr)
        tokens = s_expr.split()
        return self.build_tree(tokens)

    def build_tree(self, tokens):
        """Chuyển đổi danh sách token thành cây lồng nhau."""
        if not tokens:
            return None
        token = tokens.pop(0)
        if token == "(":
            sub_expr = []
            while tokens[0] != ")":
                sub_expr.append(self.build_tree(tokens))
            tokens.pop(0)  # Bỏ dấu ")"
            return sub_expr
        elif token == ")":
            raise ValueError("Unexpected ')'")
        else:
            return token

    def process_join(self, expr, target_var):
        """
        Xử lý JOIN, tạo triple SPARQL.
        """
        triples = []
        if not isinstance(expr, list):
            return expr, triples

        if expr[0] == "AND":
            # Xử lý từng JOIN trong AND riêng lẻ
            for sub_expr in expr[1:]:
                _, sub_triples = self.process_join(sub_expr, target_var)
                triples.extend(sub_triples)
            return target_var, triples

        if expr[0] == "JOIN":
            right_expr = expr[2]
            right_triples = []
            if isinstance(right_expr, list) and right_expr[0] == "JOIN":
                right_var, right_triples = self.process_join(right_expr, self.get_new_var())
            else: 
                right_var = right_expr
            
            # Xử lý nhánh trái
            left_expr = expr[1]
            if isinstance(left_expr, list) and left_expr[0] == "R":
                rel = left_expr[1]
                right = right_var
                left = target_var
                if right[0] != '?':
                    right = "wd:" + right
                if left[0] !='?':
                    left = "wd:" + left   
                triples.append([right, f"wdt:{rel}", left])
            else:
                right = right_var
                left = target_var
                if right[0] != '?':
                    right = "wd:" + right
                if left[0] !='?':
                    left = "wd:" + left   
                triples.append([left, f"wdt:{left_expr}", right])

            # Thêm các triples từ nhánh phải trước khi thêm triple chính
            triples = right_triples + triples
            return target_var, triples

        return expr, triples

    def s_expr_to_sparql(self, s_expr):
        """Chuyển đổi từ S-Expression sang SPARQL."""
        parsed_expr = self.parse_s_expr(s_expr)
        target_var = "?answer"
        final_var, triples = self.process_join(parsed_expr, target_var)

        sparql_body = "\n  ".join([" ".join(t) + " ." for t in triples])
        sparql_query = f"""PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
SELECT DISTINCT {target_var} WHERE {{ 
  {sparql_body}
}}"""
        return sparql_query


def execute_query_with_odbc(sparql_query):
    """
    Thực thi truy vấn SPARQL trên Wikidata endpoint và trả về kết quả.
    """
    ENDPOINT_URL = "https://query.wikidata.org/sparql"
    if sparql_query == None or sparql_query == []:
        return []
    sparql = SPARQLWrapper(ENDPOINT_URL)
    sparql.setQuery(str(sparql_query))  # Ép kiểu về str cho chắc chắn
    sparql.setReturnFormat(JSON)
    
    try:
        response = sparql.query().convert()
        answers = [item["answer"]["value"] for item in response["results"]["bindings"] if "answer" in item]
        return answers
    except Exception as e:
        return []



In [20]:

def load_jsonl(file_path):
    """Đọc file JSONL và trả về danh sách các object"""


    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read().replace('(EXPECTED RESULT)', 'null').replace('(QUESTION)', 'null')

    try:
        data = ujson.loads(content)
        print("JSON loaded successfully!")
    except Exception as e:
        print(f"Error: {e}")
    return data


In [21]:
predictions = load_jsonl("LLMs/beam_prediction/generated_predictions_beam.json")
gold_data = load_jsonl("Data/LC-QuAD2.0/label_map/LC-QuAD2.0_test.json")


JSON loaded successfully!
JSON loaded successfully!


In [22]:
simcse_model = model  # Mô hình SimCSE của bạn
facc1_index = surface_index

In [23]:
gold_entity_map = {}
gold_relation_map = {}

In [24]:
query = "( AND ( JOIN [ occupation ] [ singer ] ) ( JOIN ( R [ voice actor ] ) [ South Park ] ) )"

In [25]:
set_query = convert_normed_to_s_expression(query, gold_relation_map, gold_entity_map, simcse_model, facc1_index)
set_query

occupation
voice actor
singer
South Park


['( AND ( JOIN occupation singer ) ( JOIN ( R voice actor ) Q16538 ) )']